# Valoración de Activos - Recomendación para un perfil Adverso Al Riesgo - Finanzas III 
📎📈📊

In [ ]:
# Profe, instale esto para que el código corra correctamente: pip install yfinance pandas numpy matplotlib seaborn statsmodels

#Integrantes:

# - Aylen Alvarado
# - Solange Donoso
# - Alan Espinoza
# - Sebastián Pizarro 

#Profesor:

# - Sergio Álvarez 



In [20]:
# === ANALISIS FINANCIERO PARA EMPRESAS CHILENAS ===
# Descarga de datos, estadisticas descriptivas, retornos, betas y portafolios

import yfinance as yf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import matplotlib.dates as mdates


# ----------------------------------
# CONFIGURACION INICIAL
# ----------------------------------
tickers = {
    'AGUAS-A': 'AGUAS-A.SN',
    'ANDINA-B': 'ANDINA-B.SN',
    'BSANTANDER': 'BSANTANDER.SN',
    'CAP': 'CAP.SN',
    'CENCOSUD': 'CENCOSUD.SN',
    'CMPC': 'CMPC.SN',
    'CONCHATORO': 'CONCHATORO.SN',
    'ECL': 'ECL.SN',
    'ENELCHILE': 'ENELCHILE.SN',
    'FALABELLA': 'FALABELLA.SN'
}

start_date = '2014-01-01'
end_date = '2024-12-31'

# ----------------------------------
# DESCARGAR DATOS
# ----------------------------------

print("\nDescargando precios...")
data = yf.download(list(tickers.values()), start=start_date, end=end_date)

if isinstance(data.columns, pd.MultiIndex):
    close_prices = data['Close']
else:
    close_prices = data.to_frame(name='Close')

close_prices.columns = tickers.keys()
close_prices = close_prices.sort_index()
close_prices.to_csv("precios_empresas_chilenas.csv")

# ----------------------------------
# CARGAR IPSA
# ----------------------------------
print("Cargando IPSA desde archivo local...")
ipsa_df = pd.read_csv("S&P IPSA.csv", sep=",", quotechar='"', parse_dates=["Date"], dayfirst=True)
ipsa = ipsa_df[['Date', 'Price']].dropna()
ipsa = ipsa.set_index('Date')
ipsa['Price'] = ipsa['Price'].str.replace(',', '')
ipsa['Price'] = ipsa['Price'].astype(float)
ipsa = ipsa.sort_index()
ipsa = ipsa['Price']
ipsa.name = 'IPSA'

# ----------------------------------
# ESTADISTICA DESCRIPTIVA
# ----------------------------------
print("\nCalculando estadística descriptiva...")
descriptive_stats = close_prices.describe().T
descriptive_stats['skewness'] = close_prices.skew()
descriptive_stats['kurtosis'] = close_prices.kurt()
descriptive_stats.to_csv("estadistica_descriptiva.csv")

# ----------------------------------
# RETORNOS
# ----------------------------------
print("Calculando retornos...")
daily_returns = close_prices.pct_change().dropna()
annual_returns = (1 + daily_returns).resample('Y').prod() - 1
close_prices.to_csv("precios_diarios_ordenados.csv")
daily_returns.to_csv("retornos_diarios.csv")
annual_returns.to_csv("retornos_anuales.csv")
descriptive_stats['mediana'] = daily_returns.median()
descriptive_stats.to_csv("estadistica_descriptiva_extendida.csv")

# ----------------------------------
# BETAS
# ----------------------------------
print("Calculando betas...")
ret_ip = ipsa.pct_change().dropna()
betas = {}

for col in daily_returns.columns:
    df = pd.concat([daily_returns[col], ret_ip], axis=1).dropna()
    df.columns = ['Ri', 'Rm']
    X = sm.add_constant(df['Rm'])
    model = sm.OLS(df['Ri'], X).fit()
    betas[col] = model.params['Rm']

betas_df = pd.DataFrame.from_dict(betas, orient='index', columns=['Beta'])
betas_df.to_csv("betas.csv")

# ----------------------------------
# PORTAFOLIOS
# ----------------------------------
print("Construyendo portafolios...")
mean_returns = daily_returns.mean() * 252
cov_matrix = daily_returns.cov() * 252
correlation_matrix = daily_returns.corr()

n_portfolios = 20000
results = {'Return': [], 'Volatility': [], 'Sharpe': [], 'Weights': []}

for _ in range(n_portfolios):
    weights = np.random.random(len(tickers))
    weights /= np.sum(weights)
    port_return = np.dot(weights, mean_returns)
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = port_return / port_volatility
    results['Return'].append(port_return)
    results['Volatility'].append(port_volatility)
    results['Sharpe'].append(sharpe_ratio)
    results['Weights'].append(weights)
    port_daily_returns = daily_returns.dot(weights)
    port_median_annual = port_daily_returns.median() * 252
    results.setdefault('MedianReturn', []).append(port_median_annual)
    
portfolios_df = pd.DataFrame(results)
portfolios_df.to_csv("portafolios_simulados_con_mediana.csv")

# Identificar el portafolio con mejor Sharpe 
max_sharpe_idx = portfolios_df['Sharpe'].idxmax()
portafolio_optimo = portfolios_df.iloc[max_sharpe_idx]

print("\nPortafolio recomendado (máximo Sharpe):")
print(portafolio_optimo)

# Guardar portafolio óptimo en CSV
portafolio_optimo.to_csv("portafolio_recomendado.csv")

# ----------------------------------
# GRAFICOS ADICIONALES
# ----------------------------------
print("Generando gráficos adicionales...")

# 1. Comparación Portafolio Conservador vs IPSA
portafolio_conservador_ret = daily_returns.dot(results['Weights'][np.argmax(results['Sharpe'])])
portafolio_conservador_ret.name = 'Portafolio Conservador'
portafolio_conservador_idx = (1 + portafolio_conservador_ret).cumprod() * 100
ipsa_idx = (1 + ret_ip).cumprod() * 100
comparacion_df = pd.concat([portafolio_conservador_idx, ipsa_idx], axis=1).dropna()
comparacion_df.columns = ['Portafolio Conservador', 'IPSA']

plt.figure(figsize=(12, 6))
plt.plot(comparacion_df.index, comparacion_df['Portafolio Conservador'], label='Portafolio Conservador', linewidth=2)
plt.plot(comparacion_df.index, comparacion_df['IPSA'], label='IPSA', linewidth=2, linestyle='--', color='black')
plt.title("Crecimiento Acumulado: Portafolio Conservador vs IPSA (Base 100)")
plt.xlabel("Fecha")
plt.ylabel("Índice de Crecimiento")
plt.legend()
plt.tight_layout()
plt.savefig("comparacion_portafolio_conservador_vs_ipsa.png")
plt.close()

# 2. Evolución acumulada acciones vs IPSA
acumulados = (1 + daily_returns).cumprod() * 100
acumulados['IPSA'] = (1 + ret_ip).cumprod() * 100
plt.figure(figsize=(12, 7))
for ticker in tickers.keys():
    plt.plot(acumulados.index, acumulados[ticker], label=ticker)
plt.plot(acumulados.index, acumulados['IPSA'], label='IPSA', linewidth=3, color='black', linestyle='--')
plt.title("Evolución Acumulada del Retorno: Acciones vs IPSA (Base 100)")
plt.ylabel("Índice de Crecimiento")
plt.xlabel("Fecha")
plt.legend()
plt.tight_layout()
plt.savefig("evolucion_precios_acciones_ipsa.png")
plt.close()

# 3. Mapa de calor de volatilidad anual
vol_anual = daily_returns.resample('Y').std() * np.sqrt(252)
plt.figure(figsize=(10, 6))
sns.heatmap(vol_anual.T, cmap="YlOrRd", annot=True, fmt=".2f", linewidths=0.5)
plt.title("Volatilidad Anual por Acción (Heatmap)")
plt.tight_layout()
plt.savefig("volatilidad_anual_heatmap.png")
plt.close()

# 4. Beta vs Volatilidad
volatilidades = daily_returns.std() * np.sqrt(252)
beta_vol = pd.DataFrame({
    'Volatilidad': volatilidades,
    'Beta': betas_df['Beta']
})
plt.figure(figsize=(8,6))
sns.scatterplot(data=beta_vol, x='Volatilidad', y='Beta', hue=beta_vol.index, s=100)
plt.axhline(1, color='gray', linestyle='--')
plt.axvline(volatilidades.mean(), color='gray', linestyle='--')
plt.title("Beta vs. Volatilidad Anual")
plt.tight_layout()
plt.savefig("beta_vs_volatilidad.png")
plt.close()

# 5. Correlación Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlación")
plt.tight_layout()
plt.savefig("correlacion_heatmap.png")
plt.close()


#6. Frontera Eficiente con Mínimo Riesgo
min_vol_idx = portfolios_df['Volatility'].idxmin()
plt.figure(figsize=(10, 7))
plt.scatter(portfolios_df['Volatility'], portfolios_df['Return'], c=portfolios_df['Sharpe'], cmap='viridis', alpha=0.7)
plt.colorbar(label='Sharpe Ratio')
plt.scatter(portfolios_df.loc[min_vol_idx, 'Volatility'], portfolios_df.loc[min_vol_idx, 'Return'], color='red', label='Min Var')
plt.scatter(volatilidades.mean(), mean_returns.mean(), color='black', marker='x', label='IPSA Aprox')
plt.title("Frontera Eficiente con Portafolio de Varianza Mínima e IPSA")
plt.xlabel("Volatilidad Anual")
plt.ylabel("Retorno Esperado Anual")
plt.legend()
plt.tight_layout()
plt.savefig("frontera_eficiente_con_min_var_y_ipsa.png")
plt.close()

#7. Generar gráfico de barras con los Betas
plt.figure(figsize=(10, 6))
sns.barplot(x=betas_df.index, y=betas_df['Beta'], palette='viridis')
plt.axhline(1, color='red', linestyle='--', label='Beta = 1 (Mercado)')
plt.title('Betas de las Acciones respecto al IPSA')
plt.ylabel('Beta')
plt.xlabel('Acción')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.savefig("betas_barplot.png")
plt.close()

#8. Retorno vs Riesgo
plt.figure(figsize=(10, 6))
plt.scatter(volatilidades, mean_returns, s=100)
for i, txt in enumerate(mean_returns.index):
    plt.annotate(txt, (volatilidades[i], mean_returns[i]))
plt.title("Retorno Esperado vs. Riesgo (Volatilidad Anual)")
plt.xlabel("Volatilidad Anual")
plt.ylabel("Retorno Esperado Anual")
plt.tight_layout()
plt.savefig("retorno_vs_riesgo_acciones.png")
plt.close()

# ----------------------------------
# RECOMENDACIÓN DE PORTAFOLIO PARA PERFIL ADVERSO AL RIESGO
# ----------------------------------

# Criterio: Menor volatilidad con Sharpe ratio mayor a la media
sharpe_mean = portfolios_df['Sharpe'].mean()
filtrados = portfolios_df[portfolios_df['Sharpe'] > sharpe_mean]
portafolio_recomendado = filtrados.loc[filtrados['Volatility'].idxmin()]

# Extraer pesos en un dataframe legible
pesos_df = pd.DataFrame(
    data=[portafolio_recomendado['Weights']],
    columns=tickers.keys()
)
pesos_df['Retorno Esperado'] = portafolio_recomendado['Return']
pesos_df['Volatilidad'] = portafolio_recomendado['Volatility']
pesos_df['Sharpe Ratio'] = portafolio_recomendado['Sharpe']
pesos_df.to_csv("portafolio_recomendado.csv", index=False)

# Imprimir resumen en consola
print("\n📌 Portafolio recomendado para perfil adverso al riesgo:")
print(pesos_df.T)

# Comparar gráficamente con IPSA
print("Generando gráfico comparativo con IPSA...")

ret_portafolio_recomendado = daily_returns.dot(portafolio_recomendado['Weights'])
ret_portafolio_recomendado.name = 'Portafolio Recomendado'
idx_portafolio = (1 + ret_portafolio_recomendado).cumprod() * 100
idx_ipsa = (1 + ret_ip).cumprod() * 100

comparacion_recomendado = pd.concat([idx_portafolio, idx_ipsa], axis=1).dropna()
comparacion_recomendado.columns = ['Portafolio Recomendado', 'IPSA']

plt.figure(figsize=(12, 6))
plt.plot(comparacion_recomendado.index, comparacion_recomendado['Portafolio Recomendado'], label='Portafolio Recomendado', linewidth=2)
plt.plot(comparacion_recomendado.index, comparacion_recomendado['IPSA'], label='IPSA', linestyle='--', color='black', linewidth=2)
plt.title("Comparación Portafolio Recomendado vs IPSA (Base 100)")
plt.xlabel("Fecha")
plt.ylabel("Índice de Crecimiento")
plt.legend()
plt.tight_layout()
plt.savefig("comparacion_portafolio_recomendado_vs_ipsa.png")
plt.close()

# ----------------------------------
# GUARDAR PORTAFOLIOS CLAVE EN CSV
# ----------------------------------

# Portafolio de máxima Sharpe
max_sharpe_idx = portfolios_df['Sharpe'].idxmax()
max_sharpe = portfolios_df.loc[max_sharpe_idx].copy()
max_sharpe_weights = pd.Series(
    [round(w, 4) for w in results['Weights'][max_sharpe_idx]],
    index=tickers.keys()
)
max_sharpe_weights.name = "Peso"
max_sharpe_weights.to_csv("portafolio_max_sharpe.csv")

# Portafolio de mínima varianza

min_var_idx = portfolios_df['Volatility'].idxmin()
min_var = portfolios_df.loc[min_var_idx].copy()
min_var_weights = pd.Series(
    [round(w, 4) for w in results['Weights'][min_var_idx]],
    index=tickers.keys()
)
min_var_weights.name = "Peso"
min_var_weights.to_csv("portafolio_min_varianza.csv")

print("✅ Portafolios clave guardados como CSV.")

# ----------------------------------
# RENDIMIENTO, VOLATILIDAD Y SHARPE DE PORTAFOLIOS CLAVE Y COMPARACION CON IPSA
# ----------------------------------

# Función para calcular métricas del portafolio dada su ponderación
def calc_metrics(weights, mean_returns, cov_matrix):
    port_return = np.dot(weights, mean_returns)
    port_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = port_return / port_volatility
    return port_return, port_volatility, sharpe_ratio

# Calcular métricas para portafolio máxima Sharpe
max_sharpe_return, max_sharpe_vol, max_sharpe_sharpe = calc_metrics(
    results['Weights'][max_sharpe_idx], mean_returns, cov_matrix
)

# Calcular métricas para portafolio mínima varianza
min_var_return, min_var_vol, min_var_sharpe = calc_metrics(
    results['Weights'][min_var_idx], mean_returns, cov_matrix
)

# Calcular métricas para IPSA (anualizados)
ipsa_return = ret_ip.mean() * 252
ipsa_vol = ret_ip.std() * np.sqrt(252)
ipsa_sharpe = ipsa_return / ipsa_vol

# Crear DataFrame comparativo
comparacion = pd.DataFrame({
    'Rendimiento Anual': [max_sharpe_return, min_var_return, ipsa_return],
    'Volatilidad Anual': [max_sharpe_vol, min_var_vol, ipsa_vol],
    'Sharpe Ratio': [max_sharpe_sharpe, min_var_sharpe, ipsa_sharpe]
}, index=['Max Sharpe', 'Min Varianza', 'IPSA'])

comparacion.to_csv("comparacion_portafolios_ipsa.csv")

print("✅ Métricas de portafolios clave y comparación con IPSA guardadas.")

import matplotlib.dates as mdates

# ----------------------------------
# 1. CÁLCULO Y GRÁFICO DEL DRAWDOWN MÁXIMO
# ----------------------------------
def max_drawdown(serie_precios):
    """Calcula el drawdown máximo de una serie de precios."""
    roll_max = serie_precios.cummax()
    drawdown = (serie_precios - roll_max) / roll_max
    max_dd = drawdown.min()
    return drawdown, max_dd

# Calcular drawdown para IPSA y portafolio óptimo (máximo Sharpe)
portafolio_optimo_weights = portfolios_df.loc[portfolios_df['Sharpe'].idxmax(), 'Weights']
portafolio_optimo_ret = daily_returns.dot(portafolio_optimo_weights)
portafolio_optimo_idx = (1 + portafolio_optimo_ret).cumprod() * 100

drawdown_ipsa, max_dd_ipsa = max_drawdown(ipsa_idx)
drawdown_portafolio, max_dd_portafolio = max_drawdown(portafolio_optimo_idx)

plt.figure(figsize=(12,6))
plt.plot(drawdown_ipsa.index, drawdown_ipsa, label=f'Drawdown IPSA (Max: {max_dd_ipsa:.2%})')
plt.plot(drawdown_portafolio.index, drawdown_portafolio, label=f'Drawdown Portafolio Óptimo (Max: {max_dd_portafolio:.2%})')
plt.title('Drawdown Máximo: IPSA vs Portafolio Óptimo')
plt.xlabel('Fecha')
plt.ylabel('Drawdown')
plt.legend()
plt.tight_layout()
plt.savefig("drawdown_ipsa_vs_portafolio.png")
plt.close()

# ----------------------------------
# 2. ANÁLISIS DE CORRELACIÓN DETALLADO
# ----------------------------------
correlation_matrix = daily_returns.corr()
plt.figure(figsize=(10,8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlación entre Acciones")
plt.tight_layout()
plt.savefig("matriz_correlacion_acciones.png")
plt.close()

# ----------------------------------
# 3. PORTAFOLIO DE MÍNIMA VARIANZA
# ----------------------------------
min_var_idx = portfolios_df['Volatility'].idxmin()
min_var_weights = portfolios_df.loc[min_var_idx, 'Weights']
min_var_return = portfolios_df.loc[min_var_idx, 'Return']
min_var_volatility = portfolios_df.loc[min_var_idx, 'Volatility']

print(f"Portafolio mínima varianza: Retorno esperado {min_var_return:.2%}, Volatilidad {min_var_volatility:.2%}")

# Guardar pesos mínimos varianza en CSV
min_var_weights_df = pd.DataFrame(min_var_weights, index=tickers.keys(), columns=['Peso'])
min_var_weights_df.to_csv("portafolio_minima_varianza.csv")

# ----------------------------------
# 4. SIMULACIONES DE ESTRÉS (Días con mayor caída IPSA)
# ----------------------------------
# Seleccionar los 10 días con mayor caída diaria IPSA
ipsa_daily_ret = ipsa.pct_change().dropna()
peores_dias = ipsa_daily_ret.nsmallest(10)

print("\n10 días con mayor caída en IPSA:")
print(peores_dias)

# Visualizar efecto de esos días en portafolio mínimo varianza
ret_min_var_portafolio = daily_returns.dot(min_var_weights)
impacto_dias_estres = ret_min_var_portafolio.loc[peores_dias.index]

print("\nImpacto en portafolio mínima varianza en esos días:")
print(impacto_dias_estres)

# ----------------------------------
# 5. GRÁFICOS DE SHARPE RATIO Y RECOMENDACIONES
# ----------------------------------
plt.figure(figsize=(10,6))
sns.histplot(portfolios_df['Sharpe'], bins=50, kde=True)
plt.axvline(portfolios_df['Sharpe'].max(), color='red', linestyle='--', label='Máximo Sharpe')
plt.title('Distribución del Sharpe Ratio de Portafolios Simulados')
plt.xlabel('Sharpe Ratio')
plt.ylabel('Frecuencia')
plt.legend()
plt.tight_layout()
plt.savefig("sharpe_ratio_histograma.png")
plt.close()


# ----------------------------------
# SHARPE RATIO: PORTAFOLIO VS IPSA
# ----------------------------------
print("\nCalculando Sharpe Ratio comparado...")

# Tasa libre de riesgo aproximada (puedes cambiarla si te dan una)
risk_free_rate = 0.05

# Retornos diarios
# Alinear los índices para evitar KeyError
common_idx = daily_returns.index.intersection(ipsa.pct_change().dropna().index)
ret_port = daily_returns.loc[common_idx].dot(weights)
ret_ip = ipsa.pct_change().dropna().loc[common_idx]

# Sharpe diario
sharpe_port = (ret_port.mean() - risk_free_rate) / ret_port.std()
sharpe_ipsa = (ret_ip.mean() - risk_free_rate) / ret_ip.std()

# Anualizado
sharpe_port_annual = sharpe_port * np.sqrt(252)
sharpe_ipsa_annual = sharpe_ipsa * np.sqrt(252)

# Mostrar
print(f"📈 Sharpe Ratio Portafolio Conservador (anual): {sharpe_port_annual:.2f}")
print(f"📉 Sharpe Ratio IPSA (anual): {sharpe_ipsa_annual:.2f}")

# Comparación Portafolio con Mayor Mediana vs IPSA
print("Generando gráfico de comparación con portafolio mediana...")

idx_median_max = portfolios_df['MedianReturn'].idxmax()
weights_median_max = portfolios_df.iloc[idx_median_max]['Weights']
portafolio_median_ret = daily_returns.dot(weights_median_max)
portafolio_median_ret.name = 'Portafolio Mediana'
portafolio_median_idx = (1 + portafolio_median_ret).cumprod() * 100
portafolio_median_idx.name = 'Portafolio Mediana'

comparacion_mediana_df = pd.concat([portafolio_median_idx, ipsa_idx], axis=1).dropna()

plt.figure(figsize=(12, 6))
plt.plot(comparacion_mediana_df.index, comparacion_mediana_df['Portafolio Mediana'], label='Portafolio Mediana', linewidth=2)
plt.plot(comparacion_mediana_df.index, comparacion_mediana_df['IPSA'], label='IPSA', linestyle='--', linewidth=2, color='black')
plt.title("Crecimiento Acumulado: Portafolio con Mayor Mediana vs IPSA (Base 100)")
plt.xlabel("Fecha")
plt.ylabel("Índice de Crecimiento")
plt.legend()
plt.tight_layout()
plt.savefig("comparacion_portafolio_mediana_vs_ipsa.png")
plt.close()

# Recomendación simple basada en Sharpe ratio y volatilidad para inversor adverso al riesgo
print(f"\nRecomendación para inversor adverso al riesgo:")
print(f"- Portafolio mínima varianza con retorno esperado {min_var_return:.2%} y volatilidad {min_var_volatility:.2%}.")
print("- Este portafolio busca minimizar la volatilidad, ideal para quien quiere menor riesgo.")
print(f"- Portafolio óptimo con máximo Sharpe ratio tiene retorno esperado {portfolios_df['Return'].max():.2%} y volatilidad {portfolios_df.loc[portfolios_df['Sharpe'].idxmax(),'Volatility']:.2%}.")
print("- Este portafolio maximiza la rentabilidad ajustada por riesgo, pero con más volatilidad.")

print("✅ Análisis adicionales, gráficos y reporte generado correctamente.")

# ----------------------------------
# EXPORTAR MATRICES Y CVAR
# ----------------------------------
correlation_matrix.to_csv("matriz_correlacion.csv")
cov_matrix.to_csv("matriz_varianza_covarianza.csv")

confidence_level = 0.95
cvar_diario = daily_returns.apply(lambda x: x[x <= x.quantile(1 - confidence_level)].mean())
cvar_diario.name = 'CVaR Diario 95%'
cvar_diario.to_csv("cvar_diario_95.csv")

# -------------------------------
# RECOMENDACION DE PORTAFOLIO PARA PERFIL ADVERSO AL RIESGO
# -------------------------------

# Índice del portafolio conservador (mínima varianza)
idx_min_var = np.argmin(portfolios_df['Volatility'])
pesos_min_var = portfolios_df.loc[idx_min_var, 'Weights']

# Beta ponderada del portafolio conservador
beta_ponderada = 0
for i, ticker in enumerate(tickers.keys()):
    beta_ponderada += pesos_min_var[i] * betas_df.loc[ticker, 'Beta']

# Retorno esperado y volatilidad del portafolio conservador
retorno_min_var = portfolios_df.loc[idx_min_var, 'Return']
vol_min_var = portfolios_df.loc[idx_min_var, 'Volatility']
sharpe_min_var = portfolios_df.loc[idx_min_var, 'Sharpe']

# Guardar recomendación en archivo txt
ruta_recomendacion = "recomendacion_portafolio_adverso.txt"
with open(ruta_recomendacion, 'w') as f:
    f.write("Recomendación para Inversionista Adverso al Riesgo\n")
    f.write("==================================================\n\n")
    f.write(f"Portafolio conservador seleccionado: Portafolio de mínima varianza\n\n")
    f.write(f"Retorno esperado anual: {retorno_min_var * 100:.2f}%\n")
    f.write(f"Volatilidad anual (riesgo): {vol_min_var * 100:.2f}%\n")
    f.write(f"Ratio de Sharpe: {sharpe_min_var:.3f}\n")
    f.write(f"Beta ponderada del portafolio: {beta_ponderada:.3f}\n\n")
    f.write("Composición del portafolio (pesos en %):\n")
    for i, ticker in enumerate(tickers.keys()):
        f.write(f" - {ticker}: {pesos_min_var[i] * 100:.2f}%\n")
    f.write("\n")
    f.write("Interpretación:\n")
    f.write("- Este portafolio busca minimizar la volatilidad, ideal para perfiles adversos al riesgo.\n")
    f.write("- Su beta ponderada menor a 1 indica menor sensibilidad a los movimientos del mercado (IPSA).\n")
    f.write("- Aunque el retorno esperado es más bajo que portafolios más arriesgados, la reducción del riesgo puede proteger el capital.\n")
    f.write("- El ratio de Sharpe muestra una buena relación retorno-riesgo para este perfil.\n")
    f.write("\nSe recomienda revisar periódicamente este portafolio para ajustarlo según las condiciones del mercado.\n")
    
    

print(f"✅ Archivo de recomendación generado en: {ruta_recomendacion}")

# ----------------------------------
# 1. Simulación de escenarios Monte Carlo para portafolio recomendado
# ----------------------------------
print("Simulando escenarios Monte Carlo para portafolio recomendado...")

n_simulaciones = 1000
dias_sim = 252  # 1 año de trading
pesos_optimos = portfolios_df.loc[portfolios_df['Sharpe'].idxmax(), 'Weights']  # Mejor portafolio Sharpe

# Simulación de retornos diarios multivariados
mean_daily_ret = daily_returns.mean()
cov_daily = daily_returns.cov()

resultados_mc = np.zeros((dias_sim, n_simulaciones))

for i in range(n_simulaciones):
    retornos_sim = np.random.multivariate_normal(mean_daily_ret, cov_daily, dias_sim)
    # Calculo de serie de precios (asumiendo base 100)
    precios_sim = 100 * np.cumprod(1 + np.dot(retornos_sim, pesos_optimos))
    resultados_mc[:, i] = precios_sim

# Graficar simulaciones
plt.figure(figsize=(12,6))
plt.plot(resultados_mc, color='grey', alpha=0.1)
plt.title('Simulación Monte Carlo: Escenarios de Precio para Portafolio Óptimo (Base 100)')
plt.xlabel('Días')
plt.ylabel('Valor Portafolio')
plt.grid(True)
plt.tight_layout()
plt.savefig("simulacion_monte_carlo_portafolio_optimo.png")
plt.close()

# ----------------------------------
# 2. Cálculo de Drawdown máximo para portafolio conservador
# ----------------------------------
print("Calculando Drawdown máximo para portafolio conservador...")

portafolio_conservador_ret = daily_returns.dot(results['Weights'][np.argmax(results['Sharpe'])])
portafolio_conservador_idx = (1 + portafolio_conservador_ret).cumprod()

running_max = portafolio_conservador_idx.cummax()
drawdown = (portafolio_conservador_idx - running_max) / running_max

max_drawdown = drawdown.min()
max_drawdown_date = drawdown.idxmin()

print(f"Drawdown máximo: {max_drawdown:.2%} en fecha {max_drawdown_date.date()}")

# Graficar Drawdown
plt.figure(figsize=(12,5))
plt.plot(drawdown, color='red')
plt.title(f'Drawdown máximo del Portafolio Conservador: {max_drawdown:.2%} en {max_drawdown_date.date()}')
plt.ylabel('Drawdown')
plt.xlabel('Fecha')
plt.grid(True)
plt.tight_layout()
plt.savefig("drawdown_maximo_portafolio_conservador.png")
plt.close()

# ----------------------------------
# 3. Análisis simple de liquidez usando Volumen promedio diario
# ----------------------------------
print("Analizando liquidez promedio diaria de activos...")

volumen_promedio = daily_returns.copy()
volumen_promedio[:] = 0  # inicializar

# Extraer volumen de los datos descargados
try:
    volumen_data = data['Volume']  # data descargada con yf.download
    volumen_promedio = volumen_data[list(tickers.values())].mean()
    volumen_promedio.index = tickers.keys()
except Exception as e:
    print("No se pudo cargar volumen, revisa que 'data' contenga la info de volumen.")
    volumen_promedio = pd.Series(index=tickers.keys(), data=np.nan)

# Mostrar tabla simple de volumen promedio
print("\nVolumen promedio diario (millones de acciones):")
print((volumen_promedio / 1e6).round(2))

# Guardar volumen promedio en CSV
volumen_promedio.to_csv("volumen_promedio_diario.csv")

plt.figure(figsize=(10,6))
sns.barplot(x=volumen_promedio.index, y=volumen_promedio.values)
plt.title("Volumen Promedio Diario por Activo")
plt.ylabel("Volumen Promedio (acciones)")
plt.xlabel("Activo")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("volumen_promedio_diario.png")
plt.close()

# Leer los pesos desde el archivo
df = pd.read_csv("portafolio_max_sharpe.csv", index_col=0)
labels = df.index.tolist()
sizes = (df['Peso'] * 100).round(2).tolist()

# Definir colores
colors = ['red', 'yellow', 'pink', 'brown', 'orange', 'blue', 'green', 'purple', 'cyan', 'gray']

# Explode para destacar la acción con mayor porcentaje
max_index = sizes.index(max(sizes))
explode = [0.1 if i == max_index else 0 for i in range(len(sizes))]

# Crear gráfico
plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, colors=colors, explode=explode,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.title('Distribución porcentual del portafolio máximo Sharpe')
plt.savefig("Distribución porcentual del portafolio máximo Sharpe.png")
plt.close()

# Leer los pesos desde el archivo
df = pd.read_csv("portafolio_min_varianza.csv", index_col=0)
labels = df.index.tolist()
sizes = (df['Peso'] * 100).round(2).tolist()

# Definir colores
colors = ['red', 'yellow', 'pink', 'brown', 'orange', 'blue', 'green', 'purple', 'cyan', 'gray']

# Explode para destacar la acción con mayor porcentaje
max_index = sizes.index(max(sizes))
explode = [0.1 if i == max_index else 0 for i in range(len(sizes))]

# Crear gráfico
plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, colors=colors, explode=explode,
        autopct='%1.1f%%', shadow=True, startangle=140)
plt.axis('equal')
plt.title('Distribución porcentual del portafolio_min_varianza')
plt.savefig("Distribución porcentual del portafolio_min_varianza.png")
plt.close()
# ----------------------------------------------------------------------------------------------------------------------------------------#
print("✅ Todos los gráficos y matrices fueron generados correctamente.")




Descargando precios...


C:\Users\alana\AppData\Local\Temp\ipykernel_52332\2982991205.py:38: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(list(tickers.values()), start=start_date, end=end_date)
[*********************100%***********************]  10 of 10 completed
C:\Users\alana\AppData\Local\Temp\ipykernel_52332\2982991205.py:53: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  ipsa_df = pd.read_csv("S&P IPSA.csv", sep=",", quotechar='"', parse_dates=["Date"], dayfirst=True)
C:\Users\alana\AppData\Local\Temp\ipykernel_52332\2982991205.py:76: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  annual_returns = (1 + daily_returns).resample('Y').prod() - 1


Cargando IPSA desde archivo local...

Calculando estadística descriptiva...
Calculando retornos...
Calculando betas...
Construyendo portafolios...

Portafolio recomendado (máximo Sharpe):
Return                                                   0.142337
Volatility                                               0.226802
Sharpe                                                   0.627582
Weights         [0.007831122744090783, 0.25970084576875985, 0....
MedianReturn                                             0.105538
Name: 17514, dtype: object
Generando gráficos adicionales...


C:\Users\alana\AppData\Local\Temp\ipykernel_52332\2982991205.py:178: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  vol_anual = daily_returns.resample('Y').std() * np.sqrt(252)
C:\Users\alana\AppData\Local\Temp\ipykernel_52332\2982991205.py:227: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=betas_df.index, y=betas_df['Beta'], palette='viridis')
C:\Users\alana\AppData\Local\Temp\ipykernel_52332\2982991205.py:242: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  plt.annotate(txt, (volatilidades[i], mean_returns[i]))



📌 Portafolio recomendado para perfil adverso al riesgo:
                         0
AGUAS-A           0.106331
ANDINA-B          0.151455
BSANTANDER        0.162586
CAP               0.041817
CENCOSUD          0.078631
CMPC              0.031072
CONCHATORO        0.244504
ECL               0.002392
ENELCHILE         0.156092
FALABELLA         0.025121
Retorno Esperado  0.084289
Volatilidad       0.183293
Sharpe Ratio      0.459856
Generando gráfico comparativo con IPSA...
✅ Portafolios clave guardados como CSV.
✅ Métricas de portafolios clave y comparación con IPSA guardadas.
Portafolio mínima varianza: Retorno esperado 8.43%, Volatilidad 18.33%

10 días con mayor caída en IPSA:
Date
2020-03-16   -0.141145
2020-03-18   -0.120857
2021-05-17   -0.093302
2020-03-12   -0.063339
2021-12-20   -0.061848
2017-11-20   -0.058558
2020-03-23   -0.058284
2019-10-21   -0.046065
2020-03-09   -0.045850
2020-05-08   -0.045330
Name: IPSA, dtype: float64

Impacto en portafolio mínima varianza en esos día